In [1]:
!pip install PyWavelets
!pip install skimage
import pandas as pd

df = pd.read_csv('../dataset/coughvid/metadata_compiled.csv')

You should consider upgrading via the 'c:\users\vince\anaconda3\python.exe -m pip install --upgrade pip' command.


  Using cached skimage-0.0.tar.gz (757 bytes)


    ERROR: Command errored out with exit status 1:
     command: 'c:\users\vince\anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\vince\\AppData\\Local\\Temp\\pip-install-8z47dd0b\\skimage_a412c50a8d8342e084e27bd024b1c57b\\setup.py'"'"'; __file__='"'"'C:\\Users\\vince\\AppData\\Local\\Temp\\pip-install-8z47dd0b\\skimage_a412c50a8d8342e084e27bd024b1c57b\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\vince\AppData\Local\Temp\pip-pip-egg-info-8tplq6ng'
         cwd: C:\Users\vince\AppData\Local\Temp\pip-install-8z47dd0b\skimage_a412c50a8d8342e084e27bd024b1c57b\
    Complete output (3 lines):
    
    *** Please install the `scikit-image` package (instead of `skimage`) ***
    
    ----------------------------------------
ERROR: Could not find a version that satisfies the requirement skimage
E

In [2]:
!pip install librosa
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from IPython import display
from scipy import signal
from scipy.io import wavfile
from scipy.signal import butter,filtfilt
from scipy.signal import cwt
from scipy.signal import hilbert
from scipy.signal import resample
from scipy.signal import decimate
from scipy.signal import spectrogram
from scipy.signal.windows import get_window

audio_length = 22050

def preprocess_cough(x, fs, cutoff = 6000, normalize = True, filter_ = True, downsample = True):
    """
    Normalize, lowpass filter, and downsample cough samples in a given data folder 
    
    Inputs: x*: (float array) time series cough signal
    fs*: (int) sampling frequency of the cough signal in Hz
    cutoff: (int) cutoff frequency of lowpass filter
    normalize: (bool) normailzation on or off
    filter: (bool) filtering on or off
    downsample: (bool) downsampling on or off
    *: mandatory input
    
    Outputs: x: (float32 array) new preprocessed cough signal
    fs: (int) new sampling frequency
    """
    
    fs_downsample = cutoff*2
    
    #Preprocess Data
    if len(x.shape)>1:
        x = np.mean(x,axis=1)                          # Convert to mono
    if normalize:
        x = x/(np.max(np.abs(x))+1e-17)                # Norm to range between -1 to 1
    if filter_:
        b, a = butter(4, fs_downsample/fs, btype='lowpass') # 4th order butter lowpass filter
        x = filtfilt(b, a, x)
    if downsample:
        x = signal.decimate(x, int(fs/fs_downsample)) # Downsample for anti-aliasing
    
    fs_new = fs_downsample

    return np.float32(x), fs_new

def segment_cough(x, fs, cough_padding=0.2, min_cough_len=0.2, th_l_multiplier = 0.1, th_h_multiplier = 2):
    """Preprocess the data by segmenting each file into individual coughs using a hysteresis comparator on the signal power
    
    Inputs:
    *x (np.array): cough signal
    *fs (float): sampling frequency in Hz
    *cough_padding (float): number of seconds added to the beginning and end of each detected cough to make sure coughs are not cut short
    *min_cough_length (float): length of the minimum possible segment that can be considered a cough
    *th_l_multiplier (float): multiplier of the RMS energy used as a lower threshold of the hysteresis comparator
    *th_h_multiplier (float): multiplier of the RMS energy used as a high threshold of the hysteresis comparator
    
    Outputs:
    *coughSegments (np.array of np.arrays): a list of cough signal arrays corresponding to each cough
    cough_mask (np.array): an array of booleans that are True at the indices where a cough is in progress"""
                
    cough_mask = np.array([False]*len(x))
    
    #Define hysteresis thresholds
    rms = np.sqrt(np.mean(np.square(x)))
    seg_th_l = th_l_multiplier * rms
    seg_th_h =  th_h_multiplier*rms

    #Segment coughs
    coughSegments = []
    padding = round(fs*cough_padding)
    min_cough_samples = round(fs*min_cough_len)
    cough_start = 0
    cough_end = 0
    cough_in_progress = False
    tolerance = round(0.01*fs)
    below_th_counter = 0
    
    for i, sample in enumerate(x**2):
        if cough_in_progress:
            if sample<seg_th_l:
                below_th_counter += 1
                if below_th_counter > tolerance:
                    cough_end = i+padding if (i+padding < len(x)) else len(x)-1
                    cough_in_progress = False
                    if (cough_end+1-cough_start-2*padding>min_cough_samples):
                        coughSegments.append(x[cough_start:cough_end+1])
                        cough_mask[cough_start:cough_end+1] = True
            elif i == (len(x)-1):
                cough_end=i
                cough_in_progress = False
                if (cough_end+1-cough_start-2*padding>min_cough_samples):
                    coughSegments.append(x[cough_start:cough_end+1])
            else:
                below_th_counter = 0
        else:
            if sample>seg_th_h:
                cough_start = i-padding if (i-padding >=0) else 0
                cough_in_progress = True
    
    return coughSegments, cough_mask

You should consider upgrading via the 'c:\users\vince\anaconda3\python.exe -m pip install --upgrade pip' command.


<h3>Preprocessing</h3>

In [3]:
def split_by_physicians(df):
    column_names = ['uuid', 'datetime', 'cough_detected', 'SNR', 'latitude', 'longitude', 
                    'age', 'gender', 'respiratory_condition', 'fever_muscle_pain', 'status', 
                    'quality', 'cough_type', 'dyspnea', 'wheezing', 'stridor', 'choking', 
                    'congestion', 'nothing', 'diagnosis', 'severity' ]
    physician_01 = df.iloc[:, 0:21]
    physician_01 = physician_01[physician_01.quality_1.notna()].reset_index(drop=True)
    physician_01.columns = column_names

    physician_02 = pd.concat([df.iloc[:, 0:11], df.iloc[:, 21:31]], axis=1)
    physician_02 = physician_02[physician_02.quality_2.notna()].reset_index(drop=True)
    physician_02.columns = column_names

    physician_03 = pd.concat([df.iloc[:, 0:11], df.iloc[:, 31:41]], axis=1)
    physician_03 = physician_03[physician_03.quality_3.notna()].reset_index(drop=True)
    physician_03.columns = column_names

    physician_04 = pd.concat([df.iloc[:, 0:11], df.iloc[:, 41:51]], axis=1)
    physician_04 = physician_04[physician_04.quality_4.notna()].reset_index(drop=True)
    physician_04.columns = column_names
    return physician_01, physician_02, physician_03, physician_04
    
def process_csv(df):
    #split by physicians
    physician_01, physician_02, physician_03, physician_04 = split_by_physicians(df)
    
    df = pd.concat([physician_01,physician_02,physician_03,physician_04]).reset_index(drop=True)  
    # drop null status
    df = df[df.status.notna()]
    # drop cough_detected < 0.8
    df = df[df.cough_detected >= 0.8 ]
    # select good quality
    df = df[df['quality'].isin(['good'])]
    # shuffle
    df = df.sample(frac=1).reset_index(drop=True) 
    #Concat together rows that have at least a physician feedback
    df = df[['uuid', 'status','cough_type', 'dyspnea', 'wheezing', 'stridor', 'choking', 'congestion', 'severity']]
    return df

In [4]:
processed_df = process_csv(df)
processed_df.head(3)
processed_df.to_csv('processed_df.csv')

In [ ]:
import soundfile as sf
from skimage.transform import resize

X_cwt = []
y_cwt = []

for x in processed_df.iterrows():
    print(x[1]['uuid'])
    audio, fs = librosa.load('../dataset/coughvid/'+ x[1]['uuid']+'.wav', mono=True)
    processed_audio, sample_rate = preprocess_cough(audio, fs)
    cough_segments, cough_mask = segment_cough(processed_audio, sample_rate, min_cough_len=0.1, cough_padding=0.1, th_l_multiplier = 0.1, th_h_multiplier = 2)

    if len(cough_segments) > 0:
        for cough in cough_segments:
            if x[1]['status'] == 'COVID-19':
                coeffs, freqs = pywt.cwt(cough, np.arange(1, 64), wavelet = 'morl')
                rescale_coeffs = resize(coeffs, (64, 64), mode = 'constant')
                X_cwt.append(rescale_coeffs)
                y_cwt.append([x[1]['uuid'], 1])
            if x[1]['status'] == 'healthy':
                coeffs, freqs = pywt.cwt(cough, np.arange(1, 64), wavelet = 'morl')
                rescale_coeffs = resize(coeffs, (64, 64), mode = 'constant')
                X_cwt.append(rescale_coeffs)
                y_cwt.append([x[1]['uuid'], 0])

In [5]:
X_cwt = np.load('X_cwt.npy')
y_cwt = np.load('y_cwt.npy')

In [6]:
X_cwt = np.array(X_cwt)
y_cwt = np.array(y_cwt)

In [ ]:
np.save('X_cwt.npy', X_cwt)
np.save('y_cwt.npy', y_cwt)

In [7]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import datasets, layers, models

def create_model():
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(64, 64,1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model



In [14]:
import random
from sklearn.metrics import accuracy_score, f1_score, zero_one_loss
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, scorer
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
final_cm=np.array([[0, 0],[0, 0]])

uuids_status_unique= pd.DataFrame(y_cwt[:,0]).drop_duplicates()
uuids = np.array(uuids_status_unique)
y_uuids = np.array([int(y) for y in pd.DataFrame(y_cwt[:]).drop_duplicates()[1]])


for train, test in skf.split(uuids, y_uuids):
    train_val = set(train)
    val= list(random.sample(train_val, 78))
    train = list(train_val - set(val))

    train_uuids = uuids[train]
    val_uuids = uuids[val]
    test_uuids = uuids[test]
    
    X_train = []
    y_train = []
    X_val = []
    y_val = []
    X_test = []
    y_test = []
    
    for index,x in enumerate(X_cwt):
        if y_cwt[index][0] in train_uuids:
            X_train.append(x)
            y_train.append(int(y_cwt[index][1]))
        elif y_cwt[index][0] in val_uuids:
            X_val.append(x)
            y_val.append(int(y_cwt[index][1]))
        elif y_cwt[index][0] in test_uuids:
            X_test.append(x)
            y_test.append(int(y_cwt[index][1]))
    
    X_train = np.array(X_train)
    X_train = X_train.reshape(X_train.shape[0], 64, 64, 1)
    y_train = np.array(y_train)
    X_val = np.array(X_val)
    X_val = X_val.reshape(X_val.shape[0], 64, 64, 1)
    y_val = np.array(y_val)
    X_test = np.array(X_test)
    X_test = X_test.reshape(X_test.shape[0], 64, 64, 1)
    y_test = np.array(y_test)
    
    model=create_model()
    model.summary()
    model.fit(X_train, y_train,
        validation_data= (X_val, y_val),
        epochs=100, callbacks=[
        EarlyStopping(
            monitor="val_loss",
            min_delta=0.0001,
            patience=7,
            verbose=0,
            mode="auto",
            baseline=None,
            restore_best_weights=False,
        )
    ])
    
    preds =  model.predict(X_test)
    y_pred = [1 if x>=0.5 else 0 for x in preds]
    final_cm += confusion_matrix(y_test, y_pred)
    print(confusion_matrix(y_test, y_pred))
    print('accuracy', accuracy_score(y_test, y_pred))
    print('f1_score', f1_score(y_test, y_pred))
    print('zero one loss', zero_one_loss(y_test, y_pred))    
        

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 62, 62, 64)        640       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 31, 31, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 29, 29, 128)       73856     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 14, 14, 128)      0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 12, 12, 256)       295168    
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 6, 6, 256)       

118/118 [==============================] - 30s 246ms/step - loss: 0.6903 - accuracy: 0.5511 - val_loss: 0.6833 - val_accuracy: 0.5793
Epoch 2/100
118/118 [==============================] - 25s 214ms/step - loss: 0.6884 - accuracy: 0.5506 - val_loss: 0.6933 - val_accuracy: 0.5416
Epoch 3/100
118/118 [==============================] - 29s 245ms/step - loss: 0.6891 - accuracy: 0.5482 - val_loss: 0.6849 - val_accuracy: 0.5793
Epoch 4/100
118/118 [==============================] - 29s 249ms/step - loss: 0.6890 - accuracy: 0.5495 - val_loss: 0.6824 - val_accuracy: 0.5793
Epoch 5/100
118/118 [==============================] - 29s 249ms/step - loss: 0.6878 - accuracy: 0.5509 - val_loss: 0.6817 - val_accuracy: 0.5793
Epoch 6/100
118/118 [==============================] - 29s 248ms/step - loss: 0.6878 - accuracy: 0.5517 - val_loss: 0.6819 - val_accuracy: 0.5793
Epoch 7/100
118/118 [==============================] - 26s 218ms/step - loss: 0.6857 - accuracy: 0.5554 - val_loss: 0.6917 - val_accurac

Epoch 5/100
121/121 [==============================] - 27s 225ms/step - loss: 0.6827 - accuracy: 0.5669 - val_loss: 0.7159 - val_accuracy: 0.4337
Epoch 6/100
121/121 [==============================] - 30s 248ms/step - loss: 0.6820 - accuracy: 0.5636 - val_loss: 0.7192 - val_accuracy: 0.4337
Epoch 7/100
121/121 [==============================] - 31s 253ms/step - loss: 0.6800 - accuracy: 0.5674 - val_loss: 0.7202 - val_accuracy: 0.4362
Epoch 8/100
121/121 [==============================] - 33s 270ms/step - loss: 0.6775 - accuracy: 0.5674 - val_loss: 0.7196 - val_accuracy: 0.4311
Epoch 9/100
121/121 [==============================] - 32s 260ms/step - loss: 0.6762 - accuracy: 0.5690 - val_loss: 0.7270 - val_accuracy: 0.4337
[[219   2]
 [191   5]]
accuracy 0.5371702637889688
f1_score 0.04926108374384237
zero one loss 0.4628297362110312
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 co

119/119 [==============================] - 35s 293ms/step - loss: 0.6824 - accuracy: 0.5347 - val_loss: 0.6858 - val_accuracy: 0.7275
Epoch 13/100
119/119 [==============================] - 35s 292ms/step - loss: 0.6804 - accuracy: 0.5378 - val_loss: 0.6731 - val_accuracy: 0.7346
Epoch 14/100
119/119 [==============================] - 33s 278ms/step - loss: 0.6793 - accuracy: 0.5399 - val_loss: 0.6825 - val_accuracy: 0.7370
Epoch 15/100
119/119 [==============================] - 34s 285ms/step - loss: 0.6752 - accuracy: 0.5431 - val_loss: 0.6877 - val_accuracy: 0.7275
[[241  11]
 [206   4]]
accuracy 0.5303030303030303
f1_score 0.035555555555555556
zero one loss 0.4696969696969697
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_32 (Conv2D)          (None, 62, 62, 64)        640       
                                                                 
 max_pooling2d_24 (MaxPoo

Epoch 9/100
120/120 [==============================] - 33s 273ms/step - loss: 0.6809 - accuracy: 0.5505 - val_loss: 0.6911 - val_accuracy: 0.5584
[[269   2]
 [197   2]]
accuracy 0.5765957446808511
f1_score 0.019704433497536946
zero one loss 0.42340425531914894
Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_40 (Conv2D)          (None, 62, 62, 64)        640       
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 31, 31, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_41 (Conv2D)          (None, 29, 29, 128)       73856     
                                                                 
 max_pooling2d_31 (MaxPoolin  (None, 14, 14, 128)      0         
 g2D)                                                   

In [15]:
final_cm

array([[2508,   52],
       [2078,   41]])